<a href="https://colab.research.google.com/github/paridhika/DDL/blob/main/Resnet50_n_params.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# According to PyTorch

In [ ]:
from collections import Counter

from torchvision.models import resnet50
import torch
from torchsummary import summary

In [ ]:
# summary(resnet50(), (3, 224, 224))

# According to TF

In [1]:
import tensorflow as tf

In [53]:
model = tf.keras.applications.resnet.ResNet50(weights=None)
model.summary();

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_11[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

# Get layers with tensors

In [56]:
from tensorflow.python.ops.math_ops import sub
# import tensorflow as tf
# model = tf.keras.applications.resnet.ResNet50(weights='imagenet');
# model.summary();
import csv  
layer_count = 0
my_map = {}
for i, layer in enumerate(model.layers):
    if "add" not in layer.name:
        layer_count += 1
        # print("Layer {}: {} ({} parameters)".format(layer_count, layer.name, layer.count_params()))
        my_map[layer.name] = layer.count_params()
  
print(len(my_map))

map_list = list(my_map.items())
map_list.reverse()
with open("tensor_layers.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(map_list)

161


# Get layers with parameters

In [58]:
from tensorflow.python.ops.math_ops import sub
# import tensorflow as tf
# model = tf.keras.applications.resnet.ResNet50(weights='imagenet');
# model.summary();
import csv  
layer_count = 0
my_map = {}
for i, layer in enumerate(model.layers):
    if len(layer.weights) > 0:
      if "add" not in layer.name:
        layer_count += 1
        # print("Layer {}: {} ({} parameters)".format(layer_count, layer.name, layer.count_params()))
        my_map[layer.name] = layer.count_params()
  
print(len(my_map))

map_list = list(my_map.items())
map_list.reverse()
with open("parameters_layers.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(map_list)

107


# Compute burst size

In [81]:
from tensorflow.python.ops.math_ops import sub
# import tensorflow as tf
# model = tf.keras.applications.resnet.ResNet50(weights='imagenet');
# model.summary();
import csv  
layer_count = 0
substring = ""
my_map = {}


for i, layer in enumerate(model.layers):
    if len(layer.weights) > 0:
      if "_" in layer.name:
        layer_count += 1
        # print("Layer {}: {} ({} parameters)".format(layer_count, layer.name, layer.count_params()))
        substring = layer.name[:layer.name.rindex("_")]
        if substring in my_map:
          my_map[substring] = my_map[substring] + layer.count_params()
        else:
          my_map[substring] = layer.count_params()
      else:
        my_map[layer.name] = layer.count_params()
  
print(len(my_map))
my_map["Layer Name"] = "Theoritical"
map_list = list(my_map.items())
map_list.reverse()
with open("my_map.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(map_list)





54


**Get table of comparison between theoritical and experimental burst sizes**

In [82]:
import csv
import pandas as pd
def generate_latex_table(filename):
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)
        table = []
        for row in reader:
            table.append(row)
        
    table_string = "\\begin{tabular}{|" + "c|"*len(header) + "}\n\\hline\n"
    table_string += " & ".join(header) + "\\\\\n\\hline\n"
    for row in table:
        table_string += " & ".join(row) + "\\\\\n\\hline\n"
    table_string += "\\end{tabular}"
    
    return table_string

numbers = [2049839.5,1048383.5,2359135.5,1048447.5,1048367.5,2359167.5,1048367.5,1048367.5,2096959.5,2359311.5,524079.5,262015.5,589647.5,261983.5,261951.5,589695.5,261951.5,262015.5,589615.5,261935.5,261935.5,589663.5,262015.5,262527.5,589615.5,261919.5,261935.5,524143.5,589599.5,130943.5,65359.5,147231.5,65375.5,65551.5,147247.5,65343.5,65391.5,147231.5,65327.5,65551.5,130879.5,147231.5,0,0,0,0,0,0,0,0,0,0,0,0]

rounded_values = [int(round(number)) for number in numbers]

# Load the existing CSV file into a pandas dataframe
df = pd.read_csv("my_map.csv")

# Create a new column in the dataframe with the rounded integer values
df["Experimental"] = rounded_values

# Save the updated dataframe to the existing CSV file
df.to_csv("my_map.csv", index=False)

filename = 'my_map.csv'
index_col = [i for i in range(1, 55)] #index column
index_col = index_col[::-1]
with open(filename, 'r') as file:
    reader = csv.reader(file)
    header = next(reader) #get header
    header.insert(0, 'Burst Count') #insert "Index" header in the first position
    data = [row for row in reader] #get data

#create new data with index column
new_data = []
for i, row in enumerate(data):
    row.insert(0, index_col[i])
    new_data.append(row)

#write data to a new file
with open('new_file.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(new_data)



table_string = generate_latex_table("new_file.csv")
with open('latex_bursts.tex', 'w') as f:
        f.write(table_string)

# Compute computation time

In [51]:
# import tensorflow as tf

# # Define your ResNet50 model
# model = tf.keras.applications.ResNet50(weights='imagenet')
import time

# Create an example input tensor
input_tensor = tf.random.normal([1,224,224,3])

# Record the start time
start = time.time()

# Perform a forward pass through the model
output = model(input_tensor)

# Record the end time
end = time.time()

# Calculate the total computation time
total_time = end - start

print("Total computation time:", total_time, "seconds")


Total computation time: 0.5615370273590088 seconds


# In summary

In [ ]:
import tensorflow as tf
model = tf.keras.applications.resnet.ResNet50(weights='imagenet');
# model.summary();
len(model.layers)
trainableLayerNameList = [layer.name for layer in model.layers if layer.trainable_variables]
print(len(trainableLayerNameList));

102967424/102967424 [==============================] - 1s 0us/step
107


In [ ]:
trainable = 0

# iterate over the layers in the model
for layer in model.layers:
    # check if the layer has trainable parameters
    if layer.trainable_variables:
        # if the layer has trainable parameters, increment the counter
        trainable += 1

# print the number of layers involved in the AllReduce operation
print(f'Number of layers with trainable parameters: {trainable}')

Number of layers with trainable parameters: 107


In [ ]:
allreduce_layers = 0;
for layer in model.layers:
    # check if the layer has trainable parameters
    if not layer.trainable_variables:
        # if the layer has trainable parameters, increment the counter
        if "add" in layer.name:
          allreduce_layers += 1
          
# print the number of layers involved in the AllReduce operation
print(f'Number of layers not involved in AllReduce: {allreduce_layers}')
print(f'Number of layers involved in AllReduce: {len(model.layers) - allreduce_layers}')

conv2_block1_add
conv2_block2_add
conv2_block3_add
conv3_block1_add
conv3_block2_add
conv3_block3_add
conv3_block4_add
conv4_block1_add
conv4_block2_add
conv4_block3_add
conv4_block4_add
conv4_block5_add
conv4_block6_add
conv5_block1_add
conv5_block2_add
conv5_block3_add
Number of layers not involved in AllReduce: 16
Number of layers involved in AllReduce: 161


In [ ]:
# import tensorflow as tf
resnet101 = tf.keras.applications.resnet.ResNet101(weights='imagenet')
len(resnet101.layers)
trainableLayerNameList = [layer.name for layer in resnet101.layers if layer.trainable_variables]
print(len(trainableLayerNameList));
allreduce_layers = 0;
for layer in resnet101.layers:
    # check if the layer has trainable parameters
    if not layer.trainable_variables:
        if "add" in layer.name:
          allreduce_layers += 1
          # print(layer.name);
print(f'Number of layers not involved in AllReduce: {allreduce_layers}')

In [ ]:
vgg16 = tf.keras.applications.vgg16.VGG16(weights=None, include_top=False)
vgg16.summary();
print(len(vgg16.layers))
trainableLayerNameList = [layer.name for layer in vgg16.layers if layer.trainable_variables]
print(len(trainableLayerNameList));
allreduce_layers = 0;
for layer in vgg16.layers:
    # check if the layer has trainable parameters
    if not layer.trainable_variables:
        if "add" in layer.name:
          allreduce_layers += 1
          # print(layer.name);
print(f'Number of layers not involved in AllReduce: {allreduce_layers}')

In [ ]:
import re
conv2d = 0;
for layer in vgg16.layers:
  if "_conv" in layer.name:
    conv2d += 1;
print(conv2d);

13


In [ ]:
print(trainableLayerNameList)

['block1_conv1', 'block1_conv2', 'block2_conv1', 'block2_conv2', 'block3_conv1', 'block3_conv2', 'block3_conv3', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block5_conv1', 'block5_conv2', 'block5_conv3']
